In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print("Name:", gpu.name, "  Type:", gpu.device_type)

    except RuntimeError as e:
        print(e)




In [ ]:
seed=2946

In [ ]:
import warnings
# warnings.filterwarnings('once')
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['PYTHONHASHSEED']=str(seed)

In [ ]:
import tensorflow as tf


tf.random.set_seed(seed)

from keras.models import Sequential, load_model, save_model
from keras.layers import Dense,Input,Reshape, Add, Flatten,ELU,RepeatVector,TimeDistributed, Bidirectional, PReLU, Concatenate, Subtract
from keras.layers import LSTM, LayerNormalization
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Activation, Embedding, multiply, Multiply
from keras.layers import LeakyReLU
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,UpSampling2D, MaxPooling1D, UpSampling1D
from tensorflow.keras import Input, Model
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tqdm import tqdm
from keras.optimizers import Nadam
from keras.models import clone_model
from keras import backend as K
import glob
import keras
from datetime import datetime
from keras.callbacks import EarlyStopping
import time
# from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from keras.callbacks import EarlyStopping
# from sklearn.metrics import r2_score
from keras.utils import plot_model
# Commented out IPython magic to ensure Python compatibility.
import pandas as pd
import numpy as np
np.random.seed(seed)
from tensorflow.keras.layers import Add, Lambda
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import stats
from keras.utils import to_categorical
from tensorflow.keras.layers import MultiHeadAttention
# from statsmodels.tsa.stattools import adfuller
# from statsmodels.tsa.stattools import pacf
%matplotlib inline
from matplotlib.pylab import rcParams
# import seaborn as sns
rcParams['figure.figsize']=15,5

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px


In [ ]:
from time import time


import pandas as pd
import random

# For plotting
from matplotlib import offsetbox
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import seaborn as sns
import plotly.graph_objects as go

%matplotlib inline
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,7)})

#For standardising the dat
from sklearn.preprocessing import StandardScaler

#PCA
from sklearn.manifold import TSNE

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

In [ ]:
def convert_to_one_hot(arr):
    """
    Convert a given array into one-hot encoding.
    
    Args:
        arr (numpy.ndarray): Input array to be converted.
    
    Returns:
        numpy.ndarray: One-hot encoded array.
    """
    # Get the unique elements in the input array
    unique_elements = np.unique(arr)
    
    # Create a dictionary to map unique elements to one-hot vectors
    one_hot_dict = {elem: np.eye(len(unique_elements))[i] for i, elem in enumerate(unique_elements)}
    
    # Map the input array to one-hot vectors
    one_hot_arr = np.array([one_hot_dict[elem] for elem in arr])
    
    return one_hot_arr

In [ ]:
def reset_random_seeds(seed):
    os.environ['PYTHONHASHSEED']=str(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
Stations = os.listdir('Clean-dataset-LMS')
# Stations.remove('.DS_Store')

In [ ]:
Stations

In [ ]:
Column = ['Date','Tem','Hum','Pressure','Rain','Light','Ax','Ay','Az','Wx','Wy','Wz','Moisture','Count']

In [ ]:
seq_len = 10

In [ ]:


def tSNE(Train, TrainL):
    """
    Perform t-SNE dimensionality reduction on the given data and visualize it in an interactive 3D scatter plot using Plotly.
    
    Args:
        Train (numpy.ndarray): Input data array.
        TrainL (numpy.ndarray): Labels for the input data array.
    """
    x_subset = Train[:]
    y_subset = [np.argmax(x) for x in TrainL[:]]

    print(np.unique(y_subset))
    if x_subset.ndim>=3:
        x_subset = x_subset.reshape((x_subset.shape[0], x_subset.shape[1] * x_subset.shape[2]))
    else:
        x_subset=x_subset.reshape((x_subset.shape[0],x_subset.shape[1]))
    
    tsne = TSNE(random_state=42, n_components=3, verbose=0, perplexity=40, n_iter=300).fit_transform(x_subset)
    
    # Create a 3D scatter plot using Scatter3d trace
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=tsne[:, 0],
        y=tsne[:, 1],
        z=tsne[:, 2],
        mode='markers',
        marker=dict(
            size=4,
            color=y_subset,
            colorscale='Spectral',
            opacity=1,
            line=dict(
                width=0.5,
                color='black'
            )
        ),
        name='t-SNE'
    ))
    fig.update_layout(
    title='Visualizing through t-SNE in 3D',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'
    ),
    width=1000,  # Set width of the plot
    height=800  # Set height of the plot
)

    # Create a separate plot with a colorbar to show the colors and their corresponding values
    colorbar = go.Figure()
    colorbar.add_trace(go.Scatter(
        x=y_subset,
        y=[0] * len(y_subset),
        mode='markers',
        marker=dict(
            size=4,
            color=y_subset,
            colorscale='Spectral',
            opacity=1,
            colorbar=dict(
                title='Class Label',
                titleside='right',
                tickvals=np.unique(y_subset),
                ticktext=[str(i) for i in np.unique(y_subset)],
                ticks='outside'
            )
        )
    ))

    colorbar.update_layout(
        title='Colorbar for t-SNE plot',
        showlegend=False,
        width=500,  # Set width of the plot
        height=400  # Set height of the plot
    )

    # Display the plots side by side
    
    fig.show()
#     colorbar.show()
    


In [ ]:
def matrix(model, Train,TrainL,Test, TestL):
    f, axes = plt.subplots(1, 2, figsize=(15, 5), sharey='row')
    Predict=[]
    True_cls=[]
    test=Train
    y=TrainL

    P=model.predict(test,verbose=0)
    Predict=np.argmax(P,axis=1)
    True_cls=np.argmax(y,axis=1)

    cm = confusion_matrix(True_cls, Predict, labels=[0,1])
    disp1 = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['No Mov','Mov'])
    disp1.plot(ax=axes[0])
    
    disp1.im_.colorbar.remove()
    train_acc = (cm[0][0]+cm[1][1])/np.sum(cm)

    Predict2=[]
    True_cls2=[]
    test=Test
    y=TestL
    P=model.predict(test,verbose=0)
    Predict2=np.argmax(P,axis=1)
    True_cls2=np.argmax(y,axis=1)

    cm = confusion_matrix(True_cls2, Predict2, labels=[0,1])
    disp2 = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['No Mov','Mov'])
    disp2.plot(ax=axes[1])
    disp2.im_.colorbar.remove()
    
    # Evaluate model accuracy on training and testing sets
    
    test_acc = (cm[0][0]+cm[1][1])/np.sum(cm)

    # Display accuracy values on subplots
    axes[0].set_title('Training Set\nAccuracy: {:.2f}%'.format(train_acc * 100))
    axes[1].set_title('Testing Set\nAccuracy: {:.2f}%'.format(test_acc * 100))


    
    plt.subplots_adjust(wspace=0.40, hspace=0.1)

    plt.show()

In [ ]:
#Rearrange the Array
def makeArray(Array):
    New=np.array(Array[0])

    for i in range(1,len(Array)):
        New = np.append(New,Array[i],axis=0)
        
    return New

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
# scaler = StandardScaler()

In [ ]:
def readData(Stations):
    
    Data, C = [], []
    
#     print(Stations)
    file = Stations
    newfile = file
    df = pd.read_csv('Clean-dataset-LMS/'+newfile, header=0, index_col=None)
    print(newfile)
    df = df.reset_index(drop=True)
    data=df[['Tem','Hum','Pressure','Rain','Light','Ax','Ay','Az','Wx','Wy','Wz','Moisture','Count']].values
    data=data.astype('float32')
    data[np.where(data[:,-1]>0),-1] = 1
    count=df['Count'].values
    count[np.where(count>0)] = 1
    count=count.astype('float32')
    d = data[1:,5:8]-data[:-1,5:8]
    d = np.where(np.abs(d) <= 0.02, 0, d)
    w = data[1:,8:11]-data[:-1,8:11]
    w[np.where(np.all(d == 0, axis=1))] = 0
    count[np.where(np.any(d != 0, axis=1))] = 1
    count[np.where(np.all(d == 0, axis=1))] = 0
    
    
#     print(np.where(np.all(d == 0, axis=1)))
#     print(np.std(w[:1000,0]),np.std(w[:1000,1]),np.std(w[:1000,2]))
#     t = 2.0
#     w[0] = np.where(np.abs(w[0]) <= t*np.std(w[:1000,0]), 0, w[0])
#     w[1] = np.where(np.abs(w[1]) <= t*np.std(w[:1000,1]), 0, w[1])
#     w[2] = np.where(np.abs(w[2]) <= t*np.std(w[:1000,2]), 0, w[2])
    data = data[1:]
    count = count[1:]
    data[:,5:8]= d
    data[:,8:11]= w

    #Normalize the data
    
    data = scaler.fit_transform(data)

    Data.append(data)
    C.append(count)
   
    return makeArray(Data), makeArray(C)


In [ ]:
Data=[[] for x in range(len(Stations))]
Count=[[] for x in range(len(Stations))]
for i in range(len(Stations)):
    Data[i], Count[i] = readData(Stations[i])

In [ ]:
for i in range(23):
    idx = i
    print(Data[idx].shape,np.unique(Data[idx],axis=0).shape)

In [ ]:
for i in range(23):
    plt.plot(Data[i][:,5:11])
    plt.plot(Count[i])
    plt.show()

In [ ]:
def make_packet(D, C, seq_len):
    Packet_data = []
    Packet_label = []
    Packet_count = []

    for i in range(len(D)-seq_len):
        Packet_data.append(D[i:i+seq_len,:])
        Packet_label.append(D[i+seq_len,:])
        Packet_count.append(C[i+seq_len-1])

    Packet_data = np.array(Packet_data)
    Packet_label = np.array(Packet_label)
    Packet_count = np.array(Packet_count).reshape(-1,1)
    
    return Packet_data, Packet_label, Packet_count

def balance_data(d1, l1, c1):
    P1 = []
    L1 = []
    C1 = []
    count = 0
    for i in range(len(c1)):
        if c1[i]>0:
            P1.append(d1[i])
            L1.append(l1[i])
            C1.append(c1[i])
            count+=1

            reset_random_seeds(seed)
    idx = np.random.permutation(len(c1))        

    P2 = []
    L2 = []
    C2 = []
    idx1 = -1
    while count!=0:
        idx1+=1
        if c1[idx[idx1]] == 0:
            P2.append(d1[idx[idx1]])
            L2.append(l1[idx[idx1]])
            C2.append(c1[idx[idx1]])
            count-=1

    #Make the movement count to 1
    #Comment this line if you want to movement count
    C1=list(np.ones((len(C1))).reshape(-1,1))
    
    P1 = P1+P2
    L1 = L1+L2
    C1 = C1+C2
    P1 = np.array(P1)
    L1 = np.array(L1)
    C1 = np.array(C1)
    
    return P1, L1, C1

def mold(D):
    T = []
    for x in D:
        for t in x:
            T.append(t)
            
    return np.array(T)

In [ ]:
def train_test(Data, Count, seq_len):
    print("Train:")
    print()
    Train = []
    Train_label = []
    Train_count = []
    S = [1,2,3,4,5,6,7,8,9,10,11,12]
    for i in S:
        D = Data[i]
        C = Count[i]
        d1, l1, c1 = make_packet(D, C, seq_len)
        P1, L1, C1 = balance_data(d1, l1, c1)
        print(C1.shape)
        Train.append(P1)
        Train_label.append(L1)
        Train_count.append(C1)

    print("Test:")
    print()
    Test = []
    Test_label = []
    Test_count = []
    S = [13,14,16,17,18,19,21,22]
    for i in S:
        D = Data[i]
        C = Count[i]
        d1, l1, c1 = make_packet(D, C, seq_len)
        P1, L1, C1 = balance_data(d1, l1, c1)
        print(C1.shape)
        Test.append(P1)
        Test_label.append(L1)
        Test_count.append(C1)

    Train = mold(Train)
    Train_label = mold(Train_label)
    Train_count = mold(Train_count)
    Test = mold(Test)
    Test_label = mold(Test_label)
    Test_count = mold(Test_count)
    
    return Train, Train_label, Train_count, Test, Test_label, Test_count

In [ ]:
Auto_Train, Auto_Train_label, Auto_Train_count, Auto_Test, Auto_Test_label, Auto_Test_count = train_test(Data, Count, seq_len)


In [ ]:
LSTM_Train, LSTM_Train_label, LSTM_Train_count, LSTM_Test, LSTM_Test_label, LSTM_Test_count = train_test(Data, Count, seq_len)
LSTM_Train_count1=[]

for x in LSTM_Train_count:
    if x[0]==0:
        LSTM_Train_count1.append([1,0])
    else:
        LSTM_Train_count1.append([0,1])
LSTM_Train_count = np.array(LSTM_Train_count1) 

LSTM_Test_count1=[]

for x in LSTM_Test_count:
    if x[0]==0:
        LSTM_Test_count1.append([1,0])
    else:
        LSTM_Test_count1.append([0,1])
LSTM_Test_count = np.array(LSTM_Test_count1)


In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Dropout, Reshape, Conv1DTranspose, Conv1D
from keras.optimizers import Adam
import numpy as np

In [ ]:
# Generate synthetic time-series data using the timeGAN model
def generate_samples(timeGAN, latent_dim, num_samples):
    noise = np.random.normal(0, 1, (num_samples, latent_dim))
    generated = timeGAN.predict(noise)
    return generated


In [ ]:
input_shape = (seq_len, 13)
output_shape = input_shape
latent_dim = 5

In [ ]:
def create_feedforward_network(input_dim, output_dim):
    # Create the feedforward network
    model = Sequential()

    # Add input layer and first hidden layer
    model.add(Dense(units=64, activation='tanh', input_shape=(input_dim,)))

    # Add additional hidden layers
    model.add(Dense(units=128, activation='tanh'))
    model.add(Dense(units=64, activation='tanh'))

    # Add output layer
    model.add(Dense(units=output_dim, activation='softmax'))


    return model

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
        
class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(query=x, value=x, key=x)
#         x = self.add([x, attn_output])
#         x = self.layernorm(x)
        return attn_output

def make_encoder_multihead(seq_len, dim, num_heads, key_dim):
    input_shape = (seq_len, dim)

    inputs = tf.keras.Input(shape=input_shape, name='L10')
    x = inputs

    # Apply MultiHeadAttention
    x = GlobalSelfAttention(num_heads=num_heads,key_dim=key_dim, dropout=0.1)(x)
    
    x = Flatten()(x)
#     x = Dense(units=seq_len* dim*2, activation='tanh')(x)
#     x = Dense(units=seq_len* dim*2, activation='tanh')(x)
#     x = Dense(units=seq_len* dim*2, activation='tanh')(x)
    x = Dense(units=seq_len*dim, activation='sigmoid')(x)
    x = Reshape((seq_len, dim))(x)


    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

def make_encoder_multihead_d(seq_len, dim, num_heads, key_dim):
    input_shape = (dim)

    inputs = tf.keras.Input(shape=input_shape, name='L11')
    x = inputs
    x = RepeatVector(seq_len)(x)

    # Apply MultiHeadAttention
    x = GlobalSelfAttention(num_heads=num_heads,key_dim=key_dim, dropout=0.1)(x)
    
    x = Flatten()(x)
#     x = Dense(units=seq_len* dim*2, activation='tanh')(x)
    x = Dense(units=seq_len* dim*2, activation='tanh')(x)
#     x = Dense(units=seq_len* dim*2, activation='tanh')(x)
    x = Dense(units=seq_len*dim, activation='sigmoid')(x)
    x = Reshape((seq_len, dim))(x)


    model = tf.keras.Model(inputs=inputs, outputs=x)
    return model

In [ ]:
from tensorflow.keras.layers import Layer

In [ ]:
class ResidualConnection(Layer):
    def __init__(self, **kwargs):
        super(ResidualConnection, self).__init__(**kwargs)

    @tf.function
    def call(self, inputs):
        x, residual = inputs
        return x + residual

In [ ]:
def sigmoid_binary_crossentropy(y_true, y_pred):

    epsilon = 1e-7  # small constant to avoid numerical instability
    y_pred = K.clip(y_pred, epsilon, 1 - epsilon)  # clip y_pred to avoid log(0)
    loss = -(y_true * K.log(y_pred) + (1 - y_true) * K.log(1 - y_pred))
    return K.mean(loss, axis=-1)

def kl_divergence_loss(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true) - K.exp(y_true) + 1, axis=-1)

def wasserstein_loss(y_true, y_pred):
    return K.mean(K.abs(y_true - y_pred))

In [ ]:
def make_encoder_lstm(seq_len, latent_dim, num_layers, hidden_units):
    z = Input(shape=(seq_len, 13), name='L9')
    x = z
    for i in range(num_layers):
        if i == num_layers-1:
            return_sequences = True
        else:
            return_sequences = True
        
        x_rnn =LSTM(hidden_units, activation='tanh',  return_sequences=return_sequences, name='R3{}'.format(i+2))(x)
#         x_rnn = LayerNormalization()(x_rnn)
        if return_sequences:
            if i > 0 or x.shape[-1] == hidden_units:
                x = Add()([x, x_rnn])
            else:
                x = x_rnn
        else:
            def slice_last(x):
                return x[..., -1, :]
            x = Add()([Lambda(slice_last)(x), x_rnn])
    
    x_latent = TimeDistributed(Dense(13, activation="tanh",name='latent'))(x)
    out = x_latent
    
    return Model(z, out)




def make_encoder_cnn(seq_len, latent_dim):

    model = Sequential()
    model.add(Conv1D(32, (3), activation="tanh", padding="same"))
    model.add(MaxPooling1D((2), padding="same"))
    model.add(Conv1D(32, (3), activation="tanh", padding="same"))
    model.add(MaxPooling1D((2), padding="same"))
    model.add(Flatten())
    model.add(Dense(seq_len*13,activation="tanh"))
    model.add(Reshape((seq_len,13)))
    z = Input(shape=(seq_len,13), name='L8')
    out = model(z)
    
    return Model(z, out)

def make_decoder_cnn(seq_len, latent_dim):

    model = Sequential()
    model.add(Conv1D(32, (3), activation="tanh", padding="same"))
    model.add(MaxPooling1D((2), padding="same"))
    model.add(Conv1D(32, (3), activation="tanh", padding="same"))
    model.add(MaxPooling1D((2), padding="same"))
    model.add(Flatten())
    model.add(Dense(seq_len*13,activation="tanh"))
    model.add(Reshape((seq_len,latent_dim)))
    z = Input(shape=(seq_len,latent_dim), name='L8')
    out = model(z)
    
    return Model(z, out)

def make_decoder(seq_len, latent_dim, num_layers, hidden_units):
    z = Input(shape=(latent_dim), name='L7')
    x = RepeatVector(seq_len)(z)
    for i in range(num_layers):
        if i == 0:
            return_sequences = True
        else:
            return_sequences = True 
        
        x_rnn = LSTM(hidden_units, activation='tanh', return_sequences=return_sequences, name='R1{}'.format(i+2))(x)
#         x_rnn = LayerNormalization()(x_rnn)
        if i > 0:
            x = Add()([x, x_rnn])
        else:
            x = x_rnn
    
    x_latent = TimeDistributed(Dense(latent_dim, activation="tanh", name='latent'))(x)
    out = x_latent
    
    return Model(z, out)

def make_generator(seq_len, latent_dim, num_layers, hidden_units):
    z = Input(shape=(seq_len, 13), name='L1')
    x = z
    for i in range(num_layers):
        if i == num_layers-1:
            return_sequences = True
        else:
            return_sequences = True
        
        x_rnn = LSTM(hidden_units, activation='tanh', return_sequences=return_sequences, name='R2{}'.format(i+2))(x)
#         x_rnn = LayerNormalization()(x_rnn)
        if return_sequences:
            if i > 0 or x.shape[-1] == hidden_units:
                x = Add()([x, x_rnn])
            else:
                x = x_rnn
        else:
            def slice_last(x):
                return x[..., -1, :]
            x = Add()([Lambda(slice_last)(x), x_rnn])
    
    x_latent = TimeDistributed(Dense(13, activation="tanh", name='latent'))(x)
    out = x_latent
    
    return Model(z, out)

def make_generator2(seq_len, latent_dim, num_layers, hidden_units):
    z = Input(shape=(seq_len, 13), name='L1')
    x = z
    for i in range(num_layers):
        if i == num_layers-1:
            return_sequences = True
        else:
            return_sequences = True
        
        x_rnn = LSTM(hidden_units, activation='tanh', return_sequences=return_sequences, name='R2{}'.format(i+2))(x)
#         x_rnn = LayerNormalization()(x_rnn)
        if return_sequences:
            if i > 0 or x.shape[-1] == hidden_units:
                x = Add()([x, x_rnn])
            else:
                x = x_rnn
        else:
            def slice_last(x):
                return x[..., -1, :]
            x = Add()([Lambda(slice_last)(x), x_rnn])
    
    x_latent = TimeDistributed(Dense(13, activation="tanh",name='latent'))(x)
    out = x_latent
    
    return Model(z, out)

def make_discriminator(seq_len, latent_dim):
    model = Sequential()
    model.add(RepeatVector(seq_len))
    model.add(Bidirectional(LSTM(50, activation='tanh', input_shape=(seq_len, latent_dim), return_sequences=False, name='L6')))
    model.add(Dense(50, activation='tanh'))
    model.add(Flatten())
    model.add(Dropout(0.2))
#     model.add(Dense(13, activation='tanh'))
#     model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    z = Input(shape=(latent_dim), name='L5')
    out = model(z)
    return Model(z, out)

def make_supervisor(seq_len, latent_dim):
    model = Sequential()
#     model.add(RepeatVector(seq_len))
    model.add(LSTM(500, activation='tanh',  return_sequences=False, name='L2'))
    model.add(Dense(100, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(latent_dim))
#     model.add(Flatten())
#     model.add(Dense(latent_dim))
    z = Input(shape=(seq_len,13), name='L1')
    v = Input(shape=(seq_len,13), name='L2')
    u = Input(shape=(seq_len,13), name='L23')
    
    x = Add()([z,v])
    x = Add()([x,u])
    out = model(x)
    return Model([z, v, u], out)

def make_decoder2(seq_len, latent_dim):
    model = Sequential()
#     model.add(RepeatVector(seq_len))
    model.add(LSTM(100, activation='tanh',  input_shape = (seq_len,latent_dim), return_sequences=True, name='L2'))

    model.add(TimeDistributed(Dense(50, activation='tanh')))
    model.add(Flatten())
    model.add(Dense(seq_len*13, activation='sigmoid'))
    model.add(Reshape((seq_len,13)))
    z = Input(shape=(seq_len,latent_dim), name='L3')
    v = Input(shape=(seq_len,latent_dim), name='L4')
    
    out = model(Add()([z,v]))
    return Model([z,v], out)
    

In [ ]:
# reset_random_seeds(seed)
lstm_layer = 2
units = 50
encoder_lstm = make_encoder_lstm(seq_len, latent_dim, lstm_layer, units)
encoder_cnn = make_encoder_cnn(seq_len, latent_dim)
# decoder_cnn = make_decoder_cnn(seq_len, latent_dim)
decoder = make_decoder(seq_len, latent_dim, lstm_layer, units)
decoder2 = make_decoder2(seq_len, latent_dim)
generator = make_generator(seq_len, latent_dim, lstm_layer, units)
generator2 = make_generator(seq_len, latent_dim, lstm_layer, units)
supervisor = make_supervisor(seq_len, latent_dim)
discriminator = make_discriminator(seq_len, latent_dim)
attention_e = make_encoder_multihead(seq_len, 13, 4, 32)
attention_d = make_encoder_multihead_d(seq_len, latent_dim, 4, 32)

z = Input(shape=(seq_len, 13))
enc_tim = encoder_lstm(z)
enc_spe = encoder_cnn(z)
enc_att = attention_e(z)
sup_e = supervisor([enc_tim, enc_spe, enc_att])
dec1 = decoder(sup_e)
# dec2 = decoder_cnn(sup_e)
dec3 = attention_d(sup_e)

dec = decoder2([dec1, dec3])

autoencoder = Model(z, dec)

embedder = Model(z, sup_e)

v = Input(shape=(seq_len, 13))
gen_lstm = generator(v)
gen_spe = encoder_cnn(v)
gen_cnn = attention_e(v)
sup_z = supervisor([gen_lstm, gen_spe, gen_cnn])
disc = discriminator(sup_z)
gan = Model(v, disc)

gen_sup = Model(v, sup_z)


autoencoder.compile(loss='mse', optimizer=Adam(0.02, 0.5))
discriminator.compile(loss=sigmoid_binary_crossentropy, optimizer=Adam(0.02, 0.5), metrics=['accuracy'])
discriminator.trainable = False
# supervisor.trainable = False
encoder_cnn.trainable = False
attention_e.trainable = False
attention_d.trainable = False
# generator.compile(loss='mse', optimizer=Adam(0.0002, 0.5))



f1 = decoder(sup_z)
f2 = attention_d(sup_z)
f3 = decoder2([f1, f2])
final = Model(v, f3)
decoder.trainable = False
decoder2.trainable = False
# supervisor.trainable = False
final.compile(loss='mse', optimizer=Adam(0.02, 0.5))



# supervisor.trainable = False
gen_sup.compile(loss='mse', optimizer=Adam(0.02, 0.5))
# generator.trainable = False
gan.compile(loss=sigmoid_binary_crossentropy, optimizer=Adam(0.02, 0.5), metrics=['accuracy'])


In [ ]:
plot_model(encoder_lstm, show_shapes=True)

In [ ]:
len(LSTM_Train)

In [ ]:
batch_size = 5000
num_epochs = 5001
final_loss = 0
emb_loss = 0
auto_loss = 0
gen_his = 10
lock = True
min_loss = 0.0
max_loss = 1.0
discriminator_loss = [0,0]
generator_loss = [0,0]
steps_per_epoch = int(len(LSTM_Train)/batch_size)

# reset_random_seeds(seed)

for epoch in range(num_epochs):
    v=0
    L = np.random.permutation(len(LSTM_Train))
    for batch in range(steps_per_epoch):
        
        bs = batch_size
        if v+bs>len(LSTM_Train):
            bs = len(LSTM_Train)-v
        data = LSTM_Train[v:v+bs]
        v+=bs
        bs =len(data)
        
        
        auto_loss = autoencoder.train_on_batch(data, data)
        
        real_data_x = embedder.predict(data)
        
        noise = np.random.normal(0, 1, (bs,seq_len,13))
#         noise = scaler.fit_transform(noise)
#         noise = noise.reshape((bs, seq_len,  13))
        
    
        
        gen_sup.train_on_batch(noise,real_data_x)
        
        
        generator_loss = gan.train_on_batch(noise, np.ones((bs, 1)))
        
        
        
        if epoch % 50 == 0:
            
            
            real_loss = discriminator.train_on_batch(real_data_x, np.ones((bs,1)))
            fake_data_z = gen_sup.predict(noise)
            fake_loss = discriminator.train_on_batch(fake_data_z, np.zeros((bs,1)))
            discriminator_loss = 0.5 * np.add(real_loss, fake_loss)
            
                
                
                      
        if epoch==0 or (epoch+1)%10 == 0:
            print(real_loss[1], fake_loss[1], discriminator_loss, generator_loss)

            # Print the loss
            print('Epoch %d: Discriminator Loss=%.4f, Generator Loss=%.4f, Emb Loss=%.4f, Aut Loss=%.4f, Final Loss=%.4f' 
                  % (epoch+1, discriminator_loss[0], generator_loss[0], emb_loss, auto_loss, final_loss))
            
        if epoch%100==0:
            P = gen_sup.predict(noise)
            A= embedder.predict(LSTM_Train)
            L = np.zeros(int(len(P)/2))
            M = np.ones(len(P)-int(len(P)/2))
            L = np.hstack((L,M))
            L = convert_to_one_hot(L)
            B = np.vstack((A,P))
            zz = np.zeros((LSTM_Train_count.shape[0],1))
            zz = np.hstack((LSTM_Train_count,zz))
            cc = np.ones((L.shape[0],1))
            dd = np.zeros(L.shape)
            cc = np.hstack((dd,cc))
            ee = np.vstack((zz,cc))
            tSNE(B,ee)
        

In [ ]:
noise = np.random.normal(0, 1, (len(LSTM_Train),seq_len,13))


In [ ]:
P = gen_sup.predict(noise)


In [ ]:
Q=final.predict(noise)
Q = Q[:,-1]
print(Q[0])

In [ ]:
A = LSTM_Train[0]
print(A[0])

In [ ]:
original_data = scaler.inverse_transform(Q)

In [ ]:
print('Temp   Hum   Pres  Rain  Light  Ax   Ay   Az  Wx   Wy   Wz   Mos   Mov')
for i in range(13):
    print("%.2f "%(original_data[0][i]), end="")

In [ ]:
A= embedder.predict(LSTM_Train)

In [ ]:
A.shape, LSTM_Train_count.shape

In [ ]:
tSNE(A,LSTM_Train_count)

In [ ]:
L = np.zeros(int(len(P)/2))
M = np.ones(len(P)-int(len(P)/2))
L = np.hstack((L,M))
L = convert_to_one_hot(L)

In [ ]:
tSNE(P,L)

In [ ]:
B = np.vstack((A,P))
zz = np.zeros((LSTM_Train_count.shape[0],1))
zz = np.hstack((LSTM_Train_count,zz))
cc = np.ones((L.shape[0],1))
dd = np.zeros(L.shape)
cc = np.hstack((dd,cc))
ee = np.vstack((zz,cc))
tSNE(B,ee)
